# Project Scope
This is an MVP for Little [Color] Riding Hood Generator.



# First Initialization
First, we install the necessary libraries and import them.

In [ ]:
!pip install openai
!pip install ipywidgets
!pip install google.colab
!pip install requests
!pip install PIL
!pip install io
!pip install os
!pip install json
!pip install openai
!pip install ipywidgets
!pip install cv2
!pip install numpy


from google.colab import userdata, drive
drive.mount('/content/drive')
import requests
from PIL import Image
from io import BytesIO
import os
import json
from openai import OpenAI
import ipywidgets as widgets
from IPython.display import display
import cv2
import numpy as np

In [ ]:
import textwrap

def print_wrapped(text, width=80):
    wrapped_text = textwrap.fill(text, width=width)
    print(wrapped_text)

Now, we set up our API key and create a basic input mechanism.

In [ ]:
# OpenAI API key
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

# Initialize the OpenAI client with the API key
client = OpenAI(
    api_key=OPENAI_API_KEY
)

# Find path of JSON file in Google Drive
file_path = '/content/drive/MyDrive/green_assets/json/pantone-colors-fixed.json'

# Load Pantone colors from the JSON file
with open(file_path, 'r') as file:
    pantone_colors = json.load(file)

# Extract color names
color_names = [color["Name"] for color in pantone_colors]

# Create a dropdown for the Pantone colors
dropdown = widgets.Dropdown(
    options=color_names,
    description='Colors:',
)

# Display the dropdown
display(dropdown)

# Use a button to confirm the selection
button = widgets.Button(description="Select Color")
output = widgets.Output()

# Run Main Function
def on_button_clicked(b):
    # Access the selected color
    color = dropdown.value
    # Find the color object from the pantone_colors list
    color_obj = next((item for item in pantone_colors if item["Name"] == color), None)
    # Correctly access RGB and HEX values
    if color_obj:
        rgb_value = f"({color_obj['R']}, {color_obj['G']}, {color_obj['B']})"
        hex_value = color_obj['HexCode']
    else:
        rgb_value = "RGB not found"
        hex_value = "Hex not found"
    title = f"Little {color.capitalize()} Riding Hood"
    print(f"\nColor selected: {color} (RGB: {rgb_value}, HEX: {hex_value})")

    # Update the prompt to include RGB and HEX values
    messages = [
        {"role": "system", "content": "You are a creative assistant."},
        {"role": "user", "content": f"Imagine a tale in the whimsical and inventive universe of Bruno Munari's 'Little [Color] Riding Hood' series. This story, titled '{title}', should weave elements of adventure, caution, and curiosity, reflecting the emotional tone of '{color}' (RGB: {rgb_value}, HEX: {hex_value}). The narrative should be engaging for children, incorporating a playful yet thoughtful exploration of the world that mirrors the unique characteristics of the '{color}' cloak. Craft this tale to captivate with its creativity, humor, and moral lessons, in a style reminiscent of Munari's cherished stories."}
    ]

    # Generate the story
    response = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=messages
    )

    # Print story
    story = response.choices[0].message.content
    print(f"\nThe generated story of {title}.")
    print_wrapped(story)

    # Use OpenAI to come up with 5 image prompts based on the story

    # Message 2
    messages_2 = [
        {"role": "system", "content": "You are a creative assistant coming up with artistic visuals."},
        {"role": "user", "content": f"Please read this story, {story}, and write five detailed prompts for DALLE to generate images without any text. Each prompt should also detail the exact art style to be used; namely, use art, not realistic photos, have sketch-like textures, use vibrant yet soft color palettes, and playful line art, and have the images look like a children's book in the style of Italian Artist Bruno Munari. Keep formatting as numbered with list with each entry only having the prompt text and nothing else."}
    ]

    # Generate the prmopts
    response_2 = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=messages_2
    )

    # Print image prompts
    image_prompts = response_2.choices[0].message.content
    print("\nHere are five image prompts to help illustrate the story.")
    print_wrapped(image_prompts)

    # This splits the string into lines, then extracts the text after the number and dot.
    prompts = [line.split('. ', 1)[1] for line in image_prompts.strip().split('\n') if line]

    # Define the output path
    output_path = "/content/drive/MyDrive/green_output/dall-e_tests/jm"

    # Function to generate images for each prompt
    def generate_images(prompts, title):
        for i, prompt in enumerate(prompts, start=1):
            response = client.images.generate(
                model="dall-e-3",
                prompt=prompt,
                size="1024x1024",
                quality="standard",
                n=1,
            )
            # Assuming the response includes a URL for the generated image
            image_url = response.data[0].url
            print(f"\nGenerated image for: {prompt}\nURL: {image_url}")

            # Download the image
            response = requests.get(image_url)
            img = Image.open(BytesIO(response.content))

            # Save the image with the specified naming convention
            filename = f"{title}_Image{i}.png"
            filepath = os.path.join(output_path, filename)
            img.save(filepath)
            print(f"Image saved as {filename} in {output_path}")

    # Generate images for each prompt
    generate_images(prompts, title)

button.on_click(on_button_clicked)
display(button, output)

Dropdown(description='Colors:', options=('Egret', 'Snow White', 'Bright White', 'Cloud Dancer', 'Gardenia', 'M…

Button(description='Select Color', style=ButtonStyle())

Output()


Color selected: Quetzal Green (RGB: (0, 104, 101), HEX: #006865)

The generated story of Little Quetzal green Riding Hood.
Once upon a time, in a fantastical forest filled with vibrant colors and
enchanting creatures, there lived a curious and spirited young girl named Little
Quetzal Green Riding Hood. She was known for her radiant emerald green cloak,
woven from the feathers of the majestic Quetzal bird, a symbol of beauty and
freedom in the magical land.  Little Quetzal Green Riding Hood loved to explore
the depths of the forest, guided by her insatiable curiosity and adventurous
spirit. One sunny morning, as she skipped along the emerald-hued foliage, her
vivid cloak fluttering in the gentle breeze, she encountered a mischievous fox
named Jasper.  Jasper, with his sly smile and twinkling eyes, approached Little
Quetzal Green Riding Hood and exclaimed, "Oh, what a splendid cloak you wear,
Little Quetzal Green! Where are you headed on this fine day?"  With a twinkle in
her eye, Littl